# Pluscode 4 Built Object Service counts
Notebook to aggregate the number of BOS items in each pc4 groups

In [1]:
# imports
from pathlib import Path
from glob import glob
import pandas as pd

In [2]:
#path to csv
csv_root = Path("/Users/abharathi/Documents/gis_data/pluscodes/bos_counts/")
file_list = glob(str(csv_root/"pc4-*.csv"))
file_list

['/Users/abharathi/Documents/gis_data/pluscodes/bos_counts/pc4-substation-counts.csv',
 '/Users/abharathi/Documents/gis_data/pluscodes/bos_counts/pc4-powerline-counts.csv',
 '/Users/abharathi/Documents/gis_data/pluscodes/bos_counts/pc4-airport-counts.csv',
 '/Users/abharathi/Documents/gis_data/pluscodes/bos_counts/pc4-highway-counts.csv',
 '/Users/abharathi/Documents/gis_data/pluscodes/bos_counts/pc4-powerpole-counts.csv',
 '/Users/abharathi/Documents/gis_data/pluscodes/bos_counts/pc4-building-counts.csv',
 '/Users/abharathi/Documents/gis_data/pluscodes/bos_counts/pc4-bridge-counts.csv',
 '/Users/abharathi/Documents/gis_data/pluscodes/bos_counts/pc4-port-counts.csv']

In [3]:
substation_df = pd.read_csv(file_list[0])
substation_df.columns=['substations', 'pluscode4']
substation_df.head()

,substations,pluscode4
0,430,87G7
1,413,84VV
2,370,84CW
3,362,865Q
4,354,867W


In [4]:
powerline_df = pd.read_csv(file_list[1])
powerline_df.columns = ['powerlines', 'pluscode4']
powerline_df.head()

,powerlines,pluscode4
0,1857225,87G7
1,1644390,76X6
2,1504544,87JC
3,1404452,86JR
4,1402601,865Q


In [5]:
print(substation_df.shape, powerline_df.shape)

(932, 2) (1153, 2)


In [6]:
pd.merge(left=substation_df, right=powerline_df, how='outer', on='pluscode4')

,substations,pluscode4,powerlines
0,430.0,87G7,1857225.0
1,413.0,84VV,1230049.0
2,370.0,84CW,952791.0
3,362.0,865Q,1402601.0
4,354.0,867W,940752.0
...,...,...,...
1150,NaN,926P,30.0
1151,NaN,93R2,23.0
1152,NaN,85X7,20.0
1153,NaN,86WH,8.0


In [7]:
p=Path(file_list[0])
p.stem.split('-')

['pc4', 'substation', 'counts']

In [27]:
all_df = pd.DataFrame(columns=['pluscode4'])
for f in file_list:
    bos_name = Path(f).stem.split('-')[1]
    bos_df = pd.read_csv(f)
    bos_df.columns = [f'{bos_name}s', 'pluscode4']
    print(f'Processing {bos_name}: nrows: {bos_df.shape[0]}')
    
    all_df = pd.merge(left=all_df, right=bos_df, how='outer', on='pluscode4')
    all_df = all_df.fillna(0)
    print(f'all_df.shape: {all_df.shape}')

Processing substation: nrows: 932
all_df.shape: (932, 2)
Processing powerline: nrows: 1153
all_df.shape: (1155, 3)
Processing airport: nrows: 113
all_df.shape: (1155, 4)
Processing highway: nrows: 980
all_df.shape: (1155, 5)
Processing powerpole: nrows: 1153
all_df.shape: (1155, 6)
Processing building: nrows: 916
all_df.shape: (1156, 7)
Processing bridge: nrows: 1033
all_df.shape: (1163, 8)
Processing port: nrows: 20
all_df.shape: (1163, 9)


In [28]:
all_df['total_bos'] = all_df['substations'] + all_df['powerlines'] + all_df['airports'] \
 + all_df['highways'] + all_df['buildings'] + all_df['bridges'] + all_df['ports']

In [29]:
all_df.head(10)

,substations,pluscode4,powerlines,airports,highways,powerpoles,buildings,bridges,ports,total_bos
0,430.0,87G7,1857225.0,1.0,26647.0,1768347.0,1960423.0,5290.0,1.0,3850017.0
1,413.0,84VV,1230049.0,1.0,9995.0,1185800.0,1178711.0,1788.0,2.0,2420959.0
2,370.0,84CW,952791.0,1.0,5268.0,923015.0,782450.0,1740.0,0.0,1742620.0
3,362.0,865Q,1402601.0,1.0,15918.0,1368865.0,1151940.0,2612.0,0.0,2573434.0
4,354.0,867W,940752.0,0.0,10850.0,925994.0,409942.0,2052.0,0.0,1363950.0
5,351.0,86FQ,1139398.0,2.0,14679.0,1105467.0,870579.0,3620.0,0.0,2028629.0
6,327.0,76X6,1644390.0,2.0,14913.0,1573902.0,1434127.0,4867.0,1.0,3098627.0
7,323.0,867X,1092374.0,1.0,10652.0,1068358.0,724980.0,2289.0,0.0,1830619.0
8,321.0,86CF,1245417.0,1.0,12403.0,1204968.0,827467.0,2844.0,0.0,2088453.0
9,306.0,86HJ,1188626.0,2.0,13475.0,1130907.0,1509255.0,2703.0,0.0,2714367.0


In [30]:
all_df = all_df.fillna(0)
all_df.shape

(1163, 10)

## Get USA PC4 codes

In [11]:
from openlocationcode import openlocationcode as olc
import pluscodeplus
import shapely
import folium
import geopandas as gpd
import numpy as np
import os

In [12]:
USE_US_EXTENT=True
code_length=2

if USE_US_EXTENT:
    min_lat = 24
    max_latN = 50
    min_lon = -128
    max_lonE = -66
else:
    # extent for globe
    min_lat = 0
    max_latN = 90
    max_latS = -90
    min_lon = 0
    max_lonE = 180
    max_lonW = -180

save_path = "../data_files/"

In [13]:
def make_grid(inc:float, us_extent=False) -> (float, float):
    """Produces a regularly spaced grid of lats & lons"""
    coord_pairs = set()
    
    if us_extent:
        for lat in np.arange(min_lat, max_latN, inc):
            # eastern & northern
            for lon in np.arange(min_lon, max_lonE, inc):
                coord_pairs.add((lat, lon))
    else:
        # Northern hemisphere
        for lat in np.arange(min_lat, max_latN, inc):

            # eastern & northern
            for lon in np.arange(min_lon, max_lonE, inc):
                coord_pairs.add((lat, lon))

            # western & northern
            for lon in np.arange(min_lon, max_lonW, -inc):
                coord_pairs.add((lat, lon))

        # Southern hemisphere
        for lat in np.arange(min_lat, max_latS, -inc):

            # eastern & southern
            for lon in np.arange(min_lon, max_lonE, inc):
                coord_pairs.add((lat, lon))

            # western & southern
            for lon in np.arange(min_lon, max_lonW, -inc):
                coord_pairs.add((lat, lon))

    
    return coord_pairs

make_grid_vectorized = np.vectorize(make_grid)

In [14]:
def get_codes(coord_grid: set[tuple], code_length:int = 2) -> set[str]:
    """Given a coord grid, it returns a set of pluscode strings"""
    code_set = set()
    for e in coord_grid:
        pc = olc.encode(latitude=e[0], longitude=e[1], codeLength=code_length)

        code_set.add(pc)
    return code_set

get_codes_vectorized = np.vectorize(get_codes)

In [16]:
grid_0_5deg = make_grid(inc=0.5, us_extent=True)
print(len(grid_0_5deg))
# grid_0_5deg

6448


In [17]:
%%time
code_length=4
code_4l = get_codes(grid_0_5deg, code_length=code_length)
print(len(code_4l))

1612
CPU times: user 58.8 ms, sys: 3.01 ms, total: 61.8 ms
Wall time: 61.7 ms


In [18]:
%%time
# convert to coordinates
code_4l_gdf = pluscodeplus.geometry.get_pluscode_geometry(code_4l)
code_4l_gdf['trimmed_pluscodes'] = code_4l_gdf['pluscode'].apply(lambda row: row[:code_length])
code_4l_gdf

CPU times: user 57.2 ms, sys: 4.95 ms, total: 62.1 ms
Wall time: 71.9 ms


,pluscode,geometry,trimmed_pluscodes
0,864J0000+,"POLYGON ((-87.00000 32.00000, -87.00000 33.000...",864J
1,76XR0000+,"POLYGON ((-83.00000 29.00000, -83.00000 30.000...",76XR
2,86W80000+,"POLYGON ((-93.00000 48.00000, -93.00000 49.000...",86W8
3,84XQ0000+,"POLYGON ((-124.00000 49.00000, -124.00000 50.0...",84XQ
4,86FG0000+,"POLYGON ((-89.00000 39.00000, -89.00000 40.000...",86FG
...,...,...,...
1607,859V0000+,"POLYGON ((-102.00000 37.00000, -102.00000 38.0...",859V
1608,87Q90000+,"POLYGON ((-72.00000 45.00000, -72.00000 46.000...",87Q9
1609,879G0000+,"POLYGON ((-69.00000 37.00000, -69.00000 38.000...",879G
1610,858C0000+,"POLYGON ((-111.00000 36.00000, -111.00000 37.0...",858C


## Merge with BOS counts DF

In [33]:
merged_gdf = pd.merge(left=code_4l_gdf, right=all_df, how='left', left_on='trimmed_pluscodes', right_on='pluscode4')
merged_gdf = merged_gdf.drop(labels=['trimmed_pluscodes'], axis=1)
merged_gdf.shape

(1612, 12)

In [41]:
merged_gdf.sort_values(by='total_bos', ascending=False).head()

,pluscode,geometry,substations,pluscode4,powerlines,airports,highways,powerpoles,buildings,bridges,ports,total_bos
821,87G70000+,"POLYGON ((-74.00000 40.00000, -74.00000 41.000...",430.0,87G7,1857225.0,1.0,26647.0,1768347.0,1960423.0,5290.0,1.0,3850017.0
81,76X60000+,"POLYGON ((-95.00000 29.00000, -95.00000 30.000...",327.0,76X6,1644390.0,2.0,14913.0,1573902.0,1434127.0,4867.0,1.0,3098627.0
1592,87G80000+,"POLYGON ((-73.00000 40.00000, -73.00000 41.000...",230.0,87G8,1080468.0,3.0,13787.0,1008220.0,1912806.0,2232.0,0.0,3009526.0
442,85540000+,"POLYGON ((-117.00000 33.00000, -117.00000 34.0...",213.0,8554,1392413.0,1.0,8006.0,1341623.0,1429886.0,2029.0,0.0,2832548.0
1259,86JR0000+,"POLYGON ((-83.00000 42.00000, -83.00000 43.000...",300.0,86JR,1404452.0,1.0,11271.0,1347895.0,1387180.0,2358.0,0.0,2805562.0


In [35]:
# plot on map
m3 = folium.Map(location=[0,0], zoom_start=2)
tt = folium.GeoJsonTooltip(fields=['pluscode4', 'total_bos'])
folium.GeoJson(data=merged_gdf, tooltip=tt).add_to(m3)
m3

## Save to geojson

In [36]:
merged_geojson = merged_gdf.to_json()

In [40]:
with open(f'{save_path}/usa_bos_counts_pc4.geojson', mode='w') as file_handle:
    file_handle.write(merged_geojson)